In [42]:
# Import libraries
import sqlalchemy
import pandas as pd
from  psycopg2 import connect
import os
from config import *
from sqlalchemy import Column, ForeignKey,Table, Integer, String, Float, Boolean, MetaData, create_engine

In [43]:
# Input file paths
cleaned_tweet_file=os.path.join("","..","Resources","outputData","tweetCleandata.csv")

In [44]:
# Load tweet csv file
tweet_clean_df= pd.read_csv(cleaned_tweet_file)
tweet_clean_df.head()

,Date,TweetID,Tweet,Matched Keywords,User,Source,Followers,Friends,Favorite,OrgTweet,Sentiment
0,8/10/2020,1292795662485130000,even right certain kind liberal deeply wants g...,Trump,MenshevikM,Twitter Web App,5882.0,320.0,0.0,They're even right that there's a certain kind...,Positive
1,8/10/2020,1292795661809850000,press people encouraged voters vote trump like...,Trump,balling_it,Twitter Web App,33.0,156.0,0.0,"@jonathanchait Naw, that is the press, people ...",Positive
2,8/10/2020,1292795659704240000,trump signs executive order throw rotted scrap...,Trump,laurie71,Twitter for iPhone,85.0,141.0,0.0,@sarahcpr Trump signs an executive order to th...,Negative
3,8/10/2020,1292795658747940000,sorry want real team truthful team justice tea...,Biden,bluewave4peace,Twitter for iPhone,528.0,745.0,0.0,@glennkirschner2 Sorry I want to be on a real ...,Positive
4,8/10/2020,1292795658550810000,yeah sase cowers yelps befor jumping embarrass...,Trump,OGOPer,Twitter for iPhone,2839.0,2655.0,0.0,Yeah. Until Ben Sasse cowers and yelps befor j...,Negative


In [45]:
# Create table specific data frames

In [46]:
tweet_user_df= tweet_clean_df[['Date','Matched Keywords','User','Followers','Friends','Favorite']]

In [47]:
tweet_source_df = tweet_clean_df[['Date','Matched Keywords','Source']]

In [48]:
tweet_df = tweet_clean_df[['Date','Matched Keywords','OrgTweet','Sentiment']]

In [49]:
tweet_clean_df = tweet_clean_df[['Date','Matched Keywords','Tweet','Sentiment']]

In [50]:
# Database connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbName}')

In [51]:
# Create MetaData
meta = MetaData(engine)

In [52]:
# 1. Twitter User Table
twitter_user = Table('tweetUser', meta,
    Column('id', Integer, primary_key = True), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('User', String),
    Column('Followers', Integer),
    Column('Friends', Integer),
    Column('Favorite', Integer))

meta.create_all(engine)

tweet_user_df.to_sql(name='tweetUser', con=engine, if_exists='append', index=False)

In [59]:
# 2. Twitte Source Table
twitter_source = Table('twitterSource', meta, 
    Column('id', Integer, primary_key = True), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Source', String),
    extend_existing=True)

meta.create_all(engine)

tweet_source_df.to_sql(name='twitterSource',  con=engine, if_exists='append', index=False)

In [54]:
# 3. Twitte Data Table
twitter_data = Table('tweetsData', meta,
    Column('id', Integer, primary_key = True), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('OrgTweet', String),
    Column('Sentiment', String),
    extend_existing=True)

meta.create_all(engine)

tweet_df.to_sql(name='tweetsData', con=engine, if_exists='append', index=False)

In [55]:
# 4. Twitter Clean Data Table
twitter_clean_data = Table('tweetsCleanData', meta, 
    Column('id', Integer, primary_key = True), 
    Column('Date', String),
    Column('Matched Keywords', String),
    Column('Tweet', String),
    Column('Sentiment', String))

meta.create_all(engine)

tweet_clean_df.to_sql(name='tweetsCleanData', con=engine, if_exists='append', index=False)

In [60]:
engine.table_names()

['tweetUser', 'tweetsData', 'tweetsCleanData', 'twitterSource']